In [1]:
import numpy as np
import pandas as pd
import time
from datetime import datetime, timedelta
import warnings
from utils import *
import scipy.stats as stats
import pymc3 as pm
import arviz as az
warnings.filterwarnings('ignore')

## Flipping a coin

Imagine we tossed a biased coin 8 times (we don’t know how biased it is), and we recorded 5 heads (H) to 3 tails (T) so far. What is the probability that the next 3 tosses will all be tails?

### 1. Frequentist Way

If we consider the probability for a single toss: E(T) = p = (3/8) = 0.375
<br>Then, the ultimate answer is E({T,T,T}) = p^3 = (3/8)^3 = 0.052

## 2. Bayesian solution with MCMC simulation using PyMC3

Probabilistic programming offers an effective way to build and solve complex models and allows us to focus more on model design, evaluation, and interpretation, and less on mathematical or computational details.

Bayesian statistics is conceptually very simple; we have the knowns and the unknowns.
We refer to the knowns as data and treat it like a constant and the unknowns as parameters and treat them as probability distributions.

We use Bayes' theorem to transform the prior probability distribution into a posterior distribution.

<i>p(theta)

<i>p(theta/y)

### PyMC3

PyMC3's base code is written using Python, and the computationally demanding parts are written using NumPy and Theano.

## Empirical: Flipping coins the PyMC3 way

Generating the data (5 heads & 3 tails)

In [2]:
Head: 0 
Tail: 1
data = [1, 0, 0, 0, 1, 0, 0, 1]

Once you have a posterior sampling, you can generate samplings from random parameterizations of the model. The pymc3.sample_posterior_predictive() method below will generate new samples the size of your original observed data.

Now, Specifying the likelihood and the prior using probability distributions

In [3]:
with pm.Model() as coin_flipping:
    p = pm.Beta('p', alpha=1, beta=1)
    pm.Bernoulli('y', p=p, observed=data)
    trace = pm.sample(10000, random_seed=2019, chains=4)
    post_pred = pm.sample_posterior_predictive(trace, samples=10000, random_seed=2019)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [p]


Sampling 4 chains for 1_000 tune and 10_000 draw iterations (4_000 + 40_000 draws total) took 31 seconds.


1. The first line of the code creates a container for our model. Everything inside the with-block will be automatically added to our_first_model
2. The second line specifies the prior
3. The third line specifies the likelihood.

<b>This is the way in which we tell PyMC3 that we want to condition for the unknown on the knowns (data). The observed values can be passed as a Python list, a tuple, a NumPy array, or a pandas DataFrame.</b>

The last line is the inference button. We are asking for 10,000 samples from the posterior and will store them in the trace object. Behind this innocent line, PyMC3 has hundreds of oompa loompas singing and baking a delicious Bayesian inference just for us!

1. The first and second lines tell us that PyMC3 has automatically assigned the NUTS sampler
2. The third line says that PyMC3 will run four chains in parallel
3. The next line is telling us which variables are being sampled by which sampler
4. Finally, the last line is a progress bar, with several related metrics indicating how fast the sampler is working

We see that a total of 44,000 samples were generated. That is because extra samples are generated to help auto-tune the sampling algorithm. The tuning phase helps PyMC3 provide a reliable sample from the posterior.

<b>We can now see how frequent the next three flips are

In [4]:
np.mean(np.sum(post_pred['y'][:,:3], axis=1) == 3)

0.0908

We then check what the results look like

### Additional Plots

In [ ]:
az.plot_trace(trace)

On the left, we have a Kernel Density Estimation (KDE) plot.

On the right, we get the individual sampled values at each step during the sampling

In [ ]:
az.summary(trace)

In [ ]:
az.plot_posterior(trace, ref_val=0.5)

### Analytical solution

<font color='green'>Help me understand the math here thanks!</font>

Since we have an analytic posterior predictive distribution (Beta-Binomial[k | n, a=4, b=6] - see https://en.wikipedia.org/wiki/Conjugate_prior#Table_of_conjugate_distributions, we can exactly calculate the probability of observing three tails in the next three flips as follows:

In [5]:
from scipy.special import comb, beta as beta_fn

n, k = 3, 3  # flips, tails
a, b = 4, 6  # 1 + observed tails, 1 + observed heads

comb(n, k) * beta_fn(n + a, n - k + b) / beta_fn(a, b)

0.09090909090909091

Useful Links:<br>
    https://stackoverflow.com/questions/54676304/how-to-build-a-bayesian-simulation-model-for-flipping-coin-three-times<br>
    https://docs.pymc.io/notebooks/getting_started.html<br>
    https://thepythonguru.com/how-to-build-probabilistic-models-with-pymc3-in-bayesian/<br>
    https://eigenfoo.xyz/bayesian-modelling-cookbook/